# Basic example: prompt + model + output parser

<img src="./../../assets/simple-pipeline.jpg">

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
llm_model = Ollama(model="mistral")
chat_model = ChatOllama(model="mistral")
output_parser = StrOutputParser()

llm_chain = prompt | llm_model
chat_chain = prompt | chat_model | output_parser

/Users/marcosf/.local/share/virtualenvs/ai-cookbook-DNsoNefS/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
llm_chain.invoke({"topic": "ice cream"})

" Why don't some types of ice cream make good secret keepers?\n\nBecause they always let things slip! (Melting, get it?)"

In [3]:
chat_chain.invoke({"topic": "ice cream"})

" Why don't some flavors of ice cream make good friends?\n\nBecause they constantly argue over being the top-ping! (Scoop it up and enjoy the double entendre!)"

## Prompt

**Works with either language model type because it defines logic both for producing `BaseMessages` and for producing a `string`.**

In [4]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [5]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [6]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

## Model

In [9]:
llm_message = llm_model.invoke(prompt_value)
llm_message

' Why don\'t some types of ice cream go to parties?\n\nBecause they are afraid of the sprinklers! (Sprinklers as in "surprise parties" and water sprinklers!)'

In [10]:
chat_message = chat_model.invoke(prompt_value)
chat_message

AIMessage(content=" Why don't some flavors of ice cream make good friends?\n\nBecause they tend to quarrel! (Like strawberry and vanilla...)\n\nBut remember, no matter the flavor, everyone can agree that ice cream is always a sweet treat!")

## Output parser

In [11]:
output_parser.invoke(llm_message)

' Why don\'t some types of ice cream go to parties?\n\nBecause they are afraid of the sprinklers! (Sprinklers as in "surprise parties" and water sprinklers!)'

In [12]:
output_parser.invoke(chat_message)

" Why don't some flavors of ice cream make good friends?\n\nBecause they tend to quarrel! (Like strawberry and vanilla...)\n\nBut remember, no matter the flavor, everyone can agree that ice cream is always a sweet treat!"

# RAG search example

<img src="./../../assets/rag-pipeline.jpg">

In [41]:
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OllamaEmbeddings(model="mistral"),
)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
  ("system", "Answer the question based only on the following context: {context}"),
  ("human", "Question: {question}")
])

setup_and_retrieval = RunnableParallel({"context": retriever, "question": RunnablePassthrough()})

chain = setup_and_retrieval | prompt | chat_model | output_parser

chain.invoke("where did harrison work?")

" Harrison worked at a company named Kensho, based on the information provided in the context. However, the context does not provide any connection between Harrison's employment and bears eating honey."